In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
import numpy as np
from pathlib import Path
from pandas import DataFrame

In [2]:
def rmse(a, b):
    return np.sqrt(mean_squared_error(a, b))

In [3]:
def load_df(folder: Path) -> DataFrame:
    return pd.read_csv(folder/"data.csv")

In [4]:
def load_data_for_model(original_data: DataFrame) -> DataFrame:
    columns_to_drop = original_data.columns[original_data.columns.str.contains("_name")]
    original_data = original_data.drop(columns=columns_to_drop)
    X = original_data.drop(columns=["item_cnt_month"])
    y = original_data["item_cnt_month"]
    return X, y

In [5]:
train_folder = Path("../data/prepared/train/")
val_folder = Path("../data/prepared/val/")

In [6]:
train_df = load_df(train_folder)
val_df = load_df(val_folder)

FileNotFoundError: [Errno 2] No such file or directory: '../data/prepared/train/data.csv'

In [ ]:
X_train, y_train = load_data_for_model(train_df)
X_val, y_val = load_data_for_model(val_df)

In [ ]:
X_train

In [ ]:
X_val

In [ ]:
gb_reg = GradientBoostingRegressor(n_estimators=100)

In [ ]:
gb_reg.fit(X_train, y_train)

In [ ]:
prediction = gb_reg.predict(X_val).clip(0, 20)

In [ ]:
prediction

In [ ]:
X_val

In [ ]:
test_range = pd.read_csv("../data/src/test.csv", index_col=["shop_id", "item_id"])
target_df = test_range.join(val_df.set_index(["shop_id", "item_id"]).assign(prediction=prediction)).assign(
    date_block_num=(24 + 9), 
    item_cnt_month=lambda df: df.item_cnt_month.fillna(0),
    prediction=lambda df: df.prediction.fillna(0),
    date_year=2015, 
    date_month=9,
)

In [ ]:
rmse(target_df.item_cnt_month, target_df.prediction)

In [ ]:
target_df

In [ ]:
target_df.reset_index(drop=True)[["ID", "item_cnt_month"]].to_csv("../submission.csv", index=False)